In the Coursera - Investment Management Course by Geneve University they introduce to methods of valuation in Module 1 (Understanding Financial Markets)

- Discounting (Net Present Value)
- Comparative Analysis (Via Multiples)

### Discounting

If you have a stream of cashflows (np.array of floats) and a discount rate (assumed risk-free rate of growth) that is constant you can discount future cashflow values to the corresponding "present value". 

- Present Value: The amount of money invested at t = 0 to recover the future value under constant growth given by the discount rate

The equation is: 

$$

NPV = \sum_{i=0}^{N}\frac{c_{i}}{(1+r)^{i}}

$$

Where $r$ is the discount rate and $c_{i}$ is the cash flow at timepoint $i$. In this case $c_{0}$ is the initial investment. If $NPV > 0$ the investment is considered favorable, $<0$ is to be avoided.

In [1]:
import numpy as np

In [ ]:
def npv(discount_rate, cash_flow):
    flow_size = len(cash_flow)
    discount_multipliers = np.array([1/((1+discount_rate)**i) for i in range(flow_size)])
    pre_sum = discount_multipliers * cash_flow
    npv = np.sum(pre_sum)
    if npv > 0:
        print(f"The net present value is {npv}, which represents a sound investment.")
    elif npv < 0:
        print(f"The net present value is {npv}, which represents a bad investment.")
    else:
        print("The net present value is 0, the investment is ambiguous under this measure.")
    
    return npv


In [ ]:
#A good deal

npv(0.1, np.array([-150,20,50,70,70]))

In [ ]:
#A bad deal

npv(0.1, np.array([100,-90,20,-90,40]))

At present, this only account for finite cashflows and disregards things like company debt, long term growth and capital costs. Under the assumption of fixed growth beyond some time horizon ($g$), and a fixed weighted average of capital cost ($r_{c}$), we can determine the *terminal value* ($TV$) assuming the cashflow is infinite and the sum under constant growth converges. (Note: In *A Random Walk Down Wall Street*, this point is highlighted as acount for profits in the future and the here-after). The equation is: 

$$
TV = \frac{1+g}{r_{c}-g}FCF_{N}
$$

Where $FCF_{N}$ is the last known *free cash flow* of the company's financial statements.

In [ ]:
def terminal_value(growth_rate,capital_cost,final_flow):
    return ((1+growth_rate)/(capital_cost - growth_rate))*final_flow

We can then discount this using the same multiplier as the final cash to give the *discounted cash flow* ($DCF$):

$$
DCF = \sum_{i=1}^{N}\frac{FCF_{i}}{(1+r_{c})^{i}} + \frac{TV}{(1+r_{c})^{N}},
$$

which is a spin on *NPV*.

In [ ]:
def dcf(growth_rate, capital_cost, free_cash_flow):
    flow_size = len(free_cash_flow)
    discount_multiplier = np.array([1/((1+capital_cost)**i) for i in range(1,flow_size+1)])
    t_v = terminal_value(growth_rate, capital_cost, free_cash_flow[-1])
    pre_sum = discount_multiplier*free_cash_flow
    return np.sum(pre_sum) + (t_v/(1+capital_cost)**(flow_size))

Lastly, we can account for exisiting debt $D$ and outstanding stock shares $s$ to derive the *enterprise value* ($EV$) and *fair stock price* ($p_{f}$)

$$
EV = DCF
$$

$$
p_{f} = \frac{EV - D}{s}
$$

In [ ]:
def fair_stock(debt,shares_outstanding,growth_rate, capital_cost, free_cash_flow):
    enterprise_value = dcf(growth_rate, capital_cost, free_cash_flow)
    return (enterprise_value - debt) / shares_outstanding